In [1]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

## Creating Song Vectors

In [2]:
# Read `tracks_df`,`tracks_feature_df` csv files
tracks_df = pd.read_csv(Path("Resources/tracks.csv"))
tracks_feature_df = pd.read_csv(Path("Resources/tracks_feature.csv"))


In [3]:
tracks_df

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,track_uri,track_href,analysis_url,duration_ms,time_signature
0,0,0.591,0.721,2,-7.960,1,0.0308,0.019700,0.000244,0.1170,0.4340,94.053,6KaZC1JpPRq9CNZ3X8jBCM,spotify:track:6KaZC1JpPRq9CNZ3X8jBCM,https://api.spotify.com/v1/tracks/6KaZC1JpPRq9...,https://api.spotify.com/v1/audio-analysis/6KaZ...,246933,4
1,1,0.399,0.955,11,-3.662,0,0.0822,0.002600,0.376000,0.4250,0.6790,156.583,7hiqCnFcOBFAqZSf3AOY8a,spotify:track:7hiqCnFcOBFAqZSf3AOY8a,https://api.spotify.com/v1/tracks/7hiqCnFcOBFA...,https://api.spotify.com/v1/audio-analysis/7hiq...,165549,4
2,2,0.573,0.915,1,-3.658,1,0.0532,0.809000,0.304000,0.3240,0.7580,105.323,1SxVwlqUaz6PdQSqjclahn,spotify:track:1SxVwlqUaz6PdQSqjclahn,https://api.spotify.com/v1/tracks/1SxVwlqUaz6P...,https://api.spotify.com/v1/audio-analysis/1SxV...,230000,4
3,3,0.492,0.801,2,-7.679,1,0.0357,0.008960,0.000120,0.1870,0.4910,74.969,1XGX6lvmT464sVpJKTF3aV,spotify:track:1XGX6lvmT464sVpJKTF3aV,https://api.spotify.com/v1/tracks/1XGX6lvmT464...,https://api.spotify.com/v1/audio-analysis/1XGX...,229600,4
4,4,0.219,0.367,0,-9.696,0,0.0285,0.072600,0.000006,0.0832,0.1070,148.622,317d1IqYa8JhV8BGVDtgwZ,spotify:track:317d1IqYa8JhV8BGVDtgwZ,https://api.spotify.com/v1/tracks/317d1IqYa8Jh...,https://api.spotify.com/v1/audio-analysis/317d...,274547,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214987,2214987,0.521,0.443,7,-8.528,0,0.0610,0.774000,0.000202,0.0359,0.4700,130.242,5tpFsLvMBaeDON1s1b1hQv,spotify:track:5tpFsLvMBaeDON1s1b1hQv,https://api.spotify.com/v1/tracks/5tpFsLvMBaeD...,https://api.spotify.com/v1/audio-analysis/5tpF...,242405,4
2214988,2214988,0.633,0.943,1,-2.459,1,0.1250,0.019500,0.001060,0.0617,0.9370,122.544,66pasDZqv03zKaxEvaM06B,spotify:track:66pasDZqv03zKaxEvaM06B,https://api.spotify.com/v1/tracks/66pasDZqv03z...,https://api.spotify.com/v1/audio-analysis/66pa...,204667,4
2214989,2214989,0.556,0.716,6,-10.431,0,0.0872,0.000144,0.000009,0.3340,0.7520,147.611,1uOIgPtrlywUhBxzerwm6G,spotify:track:1uOIgPtrlywUhBxzerwm6G,https://api.spotify.com/v1/tracks/1uOIgPtrlywU...,https://api.spotify.com/v1/audio-analysis/1uOI...,224760,4
2214990,2214990,0.240,0.303,2,-13.626,1,0.0294,0.738000,0.470000,0.1030,0.0823,99.405,3C9B1hqJ71tTxNjIjPl47Y,spotify:track:3C9B1hqJ71tTxNjIjPl47Y,https://api.spotify.com/v1/tracks/3C9B1hqJ71tT...,https://api.spotify.com/v1/audio-analysis/3C9B...,149787,4


In [4]:
tracks_feature_df

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:6KaZC1JpPRq9CNZ3X8jBCM,0.591,0.721,2,-7.960,1,0.0308,0.019700,0.000244,0.1170,0.4340,94.053,246933,4
1,spotify:track:7hiqCnFcOBFAqZSf3AOY8a,0.399,0.955,11,-3.662,0,0.0822,0.002600,0.376000,0.4250,0.6790,156.583,165549,4
2,spotify:track:1SxVwlqUaz6PdQSqjclahn,0.573,0.915,1,-3.658,1,0.0532,0.809000,0.304000,0.3240,0.7580,105.323,230000,4
3,spotify:track:1XGX6lvmT464sVpJKTF3aV,0.492,0.801,2,-7.679,1,0.0357,0.008960,0.000120,0.1870,0.4910,74.969,229600,4
4,spotify:track:317d1IqYa8JhV8BGVDtgwZ,0.219,0.367,0,-9.696,0,0.0285,0.072600,0.000006,0.0832,0.1070,148.622,274547,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214987,spotify:track:5tpFsLvMBaeDON1s1b1hQv,0.521,0.443,7,-8.528,0,0.0610,0.774000,0.000202,0.0359,0.4700,130.242,242405,4
2214988,spotify:track:66pasDZqv03zKaxEvaM06B,0.633,0.943,1,-2.459,1,0.1250,0.019500,0.001060,0.0617,0.9370,122.544,204667,4
2214989,spotify:track:1uOIgPtrlywUhBxzerwm6G,0.556,0.716,6,-10.431,0,0.0872,0.000144,0.000009,0.3340,0.7520,147.611,224760,4
2214990,spotify:track:3C9B1hqJ71tTxNjIjPl47Y,0.240,0.303,2,-13.626,1,0.0294,0.738000,0.470000,0.1030,0.0823,99.405,149787,4


## Creating Playlist Vector
* The playlist here refers to the playlist we want to continue

In [5]:
# Process the input playlist data
# Read the csv file into a pandas DataFrame
input_playlist_mean_df = pd.read_csv(Path("Resources/test_input_playlist_mean.csv"))
input_playlist_df = pd.read_csv(Path("Resources/test_input_playlist.csv"))
# Review the DataFrame
input_playlist_mean_df.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0.4548,0.7518,3.2,-6.531,0.6,0.04608,0.182572,0.136074,0.22724,0.4938,115.91,229325.8,4.0


In [6]:
input_playlist_mean_df = input_playlist_mean_df.drop('Unnamed: 0',axis=1)
input_playlist_mean_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.4548,0.7518,3.2,-6.531,0.6,0.04608,0.182572,0.136074,0.22724,0.4938,115.91,229325.8,4.0


In [7]:
input_playlist_df

,Unnamed: 0,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,spotify:track:6KaZC1JpPRq9CNZ3X8jBCM,0.591,0.721,2,-7.960,1,0.0308,0.01970,0.000244,0.1170,0.434,94.053,246933,4
1,1,spotify:track:7hiqCnFcOBFAqZSf3AOY8a,0.399,0.955,11,-3.662,0,0.0822,0.00260,0.376000,0.4250,0.679,156.583,165549,4
2,2,spotify:track:1SxVwlqUaz6PdQSqjclahn,0.573,0.915,1,-3.658,1,0.0532,0.80900,0.304000,0.3240,0.758,105.323,230000,4
3,3,spotify:track:1XGX6lvmT464sVpJKTF3aV,0.492,0.801,2,-7.679,1,0.0357,0.00896,0.000120,0.1870,0.491,74.969,229600,4
4,4,spotify:track:317d1IqYa8JhV8BGVDtgwZ,0.219,0.367,0,-9.696,0,0.0285,0.07260,0.000006,0.0832,0.107,148.622,274547,4


## Preparing for data for comparing similarity
* Compare aggregated input playlist features to song features that are not in the playlist `new_songs_feature`

In [8]:
# Get the `track_uri` for the songs in the input playlist
input_playlist_df = input_playlist_df.drop('Unnamed: 0',axis=1)

In [9]:
# Create new_songs_feature_df with songs not in the current playlist
new_songs_feature_df = tracks_feature_df[~tracks_feature_df['track_uri'].isin(input_playlist_df['track_uri'].values)]
new_songs_feature_df.reset_index(inplace=True)
new_songs_feature_df = new_songs_feature_df.drop('index',axis=1)
new_songs_feature_df

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:0Va95UdRU4I13FzO4HTFl5,0.641,0.8420,6,-5.053,0,0.0390,0.381000,0.000833,0.2030,0.8570,120.120,148173,4
1,spotify:track:0pziTZE8LPaMuxtRixjbPQ,0.156,0.0126,10,-32.286,1,0.0381,0.908000,0.071900,0.1130,0.0684,100.865,364693,4
2,spotify:track:4mRYD7cyb7NOaZZtUl70tC,0.338,0.5340,11,-15.212,1,0.0418,0.000148,0.374000,0.1250,0.3120,85.937,64153,4
3,spotify:track:0utMp2xqFJYVWrqjHjHYay,0.742,0.6800,6,-4.272,0,0.1170,0.345000,0.000000,0.1810,0.5450,103.450,226800,4
4,spotify:track:3uBk6ji6nxUC81ndX2DXsb,0.621,0.6770,3,-2.956,1,0.0289,0.085200,0.000046,0.1450,0.5450,107.055,177560,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214982,spotify:track:5tpFsLvMBaeDON1s1b1hQv,0.521,0.4430,7,-8.528,0,0.0610,0.774000,0.000202,0.0359,0.4700,130.242,242405,4
2214983,spotify:track:66pasDZqv03zKaxEvaM06B,0.633,0.9430,1,-2.459,1,0.1250,0.019500,0.001060,0.0617,0.9370,122.544,204667,4
2214984,spotify:track:1uOIgPtrlywUhBxzerwm6G,0.556,0.7160,6,-10.431,0,0.0872,0.000144,0.000009,0.3340,0.7520,147.611,224760,4
2214985,spotify:track:3C9B1hqJ71tTxNjIjPl47Y,0.240,0.3030,2,-13.626,1,0.0294,0.738000,0.470000,0.1030,0.0823,99.405,149787,4


## Scaling

In [10]:
# Scale the data
track_scaler = StandardScaler()
track_scaler.fit(new_songs_feature_df.drop('track_uri',axis=1))

StandardScaler()

In [11]:
new_songs_feature_df['track_uri']

0          spotify:track:0Va95UdRU4I13FzO4HTFl5
1          spotify:track:0pziTZE8LPaMuxtRixjbPQ
2          spotify:track:4mRYD7cyb7NOaZZtUl70tC
3          spotify:track:0utMp2xqFJYVWrqjHjHYay
4          spotify:track:3uBk6ji6nxUC81ndX2DXsb
                           ...                 
2214982    spotify:track:5tpFsLvMBaeDON1s1b1hQv
2214983    spotify:track:66pasDZqv03zKaxEvaM06B
2214984    spotify:track:1uOIgPtrlywUhBxzerwm6G
2214985    spotify:track:3C9B1hqJ71tTxNjIjPl47Y
2214986    spotify:track:20s3iOslBjQ6icWjk28Wf0
Name: track_uri, Length: 2214987, dtype: object

In [12]:
scaled_new_songs = track_scaler.transform(new_songs_feature_df.drop('track_uri',axis=1))
scaled_new_songs_df = pd.DataFrame(scaled_new_songs, columns=new_songs_feature_df.drop('track_uri',axis=1).columns)
scaled_new_songs_df['track_uri']=new_songs_feature_df['track_uri']
scaled_new_songs_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_uri
0,0.490875,0.966502,0.207598,0.818865,-1.377808,-0.436068,0.074162,-0.629619,-0.032243,1.412348,0.004286,-0.634851,0.254497,spotify:track:0Va95UdRU4I13FzO4HTFl5
1,-2.137404,-2.151290,1.331547,-4.021038,0.725790,-0.443882,1.561208,-0.426225,-0.506358,-1.508656,-0.639297,0.749803,0.254497,spotify:track:0pziTZE8LPaMuxtRixjbPQ
2,-1.151122,-0.191299,1.612534,-0.986612,0.725790,-0.411758,-1.000496,0.438384,-0.443143,-0.606353,-1.138253,-1.172162,0.254497,spotify:track:4mRYD7cyb7NOaZZtUl70tC
3,1.038207,0.357529,0.207598,0.957666,-1.377808,0.241140,-0.027420,-0.632003,-0.148138,0.256688,-0.552895,-0.132028,0.254497,spotify:track:0utMp2xqFJYVWrqjHjHYay
4,0.382492,0.346252,-0.635363,1.191548,0.725790,-0.523757,-0.760503,-0.631870,-0.337784,0.256688,-0.432401,-0.446920,0.254497,spotify:track:3uBk6ji6nxUC81ndX2DXsb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214982,-0.159421,-0.533376,0.488585,0.201281,-1.377808,-0.245060,1.183097,-0.631425,-0.912517,-0.021115,0.342606,-0.032233,0.254497,spotify:track:5tpFsLvMBaeDON1s1b1hQv
2214983,0.447522,1.346171,-1.197337,1.279876,0.725790,0.310597,-0.945890,-0.628969,-0.776604,1.708671,0.085306,-0.273569,0.254497,spotify:track:66pasDZqv03zKaxEvaM06B
2214984,0.030249,0.492856,0.207598,-0.136924,-1.377808,-0.017588,-1.000507,-0.631978,0.657858,1.023424,0.923151,-0.145074,0.254497,spotify:track:1uOIgPtrlywUhBxzerwm6G
2214985,-1.682197,-1.059649,-0.916350,-0.704745,0.725790,-0.519416,1.081515,0.713135,-0.559038,-1.457170,-0.688096,-0.624529,0.254497,spotify:track:3C9B1hqJ71tTxNjIjPl47Y


## Calculating Similarity

In [13]:
# Using cosine similarity
new_songs_feature_df['similarity'] = cosine_similarity(scaled_new_songs_df.drop('track_uri',axis=1).values, input_playlist_mean_df.values)
new_songs_df_top_100 = new_songs_feature_df.sort_values('similarity', ascending=False).head(100)
new_songs_df_top_100

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,similarity
911992,spotify:track:1tyVVpfkix098IOhUvgNSl,0.526,0.605,7,-9.441,1,0.0674,0.04000,0.08840,0.1130,0.437,117.678,4640172,4,0.998667
458263,spotify:track:6IFYFacuMge4nkvsHNmgYO,0.560,0.553,7,-10.582,1,0.1200,0.24600,0.02060,0.1050,0.350,115.098,3926267,4,0.998274
1760730,spotify:track:7Bqhue1yVfeNSY9bpafNDo,0.519,0.855,7,-6.104,1,0.0994,0.07460,0.00000,0.1410,0.799,124.539,6014408,4,0.998203
719027,spotify:track:4rDhlUqRFHbDQO712doGtf,0.509,0.534,4,-12.831,0,0.0669,0.09680,0.31500,0.2640,0.418,122.479,4761600,4,0.998046
817665,spotify:track:7y17uBXUe6kfDG4CWL1KdU,0.519,0.626,1,-8.055,1,0.0593,0.01430,0.15300,0.0951,0.468,117.144,4786275,4,0.997893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292184,spotify:track:2eILJxw0TxNIxJ7bvW5M28,0.817,0.613,2,-10.346,1,0.0886,0.02050,0.85400,0.1190,0.165,124.003,5607709,4,0.996074
533773,spotify:track:5omyPs37E3FQTkExzvDbBM,0.468,0.424,0,-10.389,1,0.0959,0.11000,0.00248,0.2110,0.112,110.175,4612320,4,0.996069
694357,spotify:track:1PcDLEd9qgoS7gcu5zJ3IW,0.725,0.609,5,-12.168,1,0.0355,0.00595,0.74000,0.0839,0.239,120.873,4649380,4,0.996051
84283,spotify:track:261yeNyxlzErFzpV83mydQ,0.588,0.751,11,-5.714,0,0.0772,0.03490,0.01050,0.1340,0.401,129.083,4884787,4,0.996049


In [14]:
# Request the audio of the song and image from Spotipy